In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
dataFrame=pd.read_csv("/kaggle/input/resume-dataset/UpdatedResumeDataSet.csv")
dataFrame.head()

In [ ]:
dataFrame['Category'].value_counts()

In [ ]:
dataFrame['Resume'][0]
    

In [ ]:
dataFrame['Resume'][500]

In [ ]:
import re

def cleanRes(txt):
    cleanText = re.sub('http\S+\s', ' ', txt)
    cleanText = re.sub('RT|cc', ' ', cleanText)
    cleanText = re.sub('#\S+\s', ' ', cleanText)
    cleanText = re.sub('@\S+', '  ', cleanText)  
    cleanText = re.sub('[%s]' % re.escape("""!"#$%&'()*+,-./:;<=>?@[\]^_`{|}~"""), ' ', cleanText)
    cleanText = re.sub(r'[^\x00-\x7f]', ' ', cleanText) 
    cleanText = re.sub('\s+', ' ', cleanText)
    return cleanText

In [ ]:
dataFrame['Resume']=dataFrame['Resume'].apply(lambda x: cleanRes(x))

In [ ]:
dataFrame['Resume'][0]

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

In [ ]:
le.fit(dataFrame['Category'])
dataFrame['Category']=le.transform(dataFrame['Category'])
dataFrame.Category

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf=TfidfVectorizer(stop_words='english')

In [ ]:
tfidf.fit(dataFrame['Resume'])
reqTxt=tfidf.transform(dataFrame['Resume'])
print(reqTxt)

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test= train_test_split(reqTxt, dataFrame['Category'], test_size=0.2, random_state=38)
X_train.shape

In [ ]:
X_test.shape

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.multiclass import OneVsRestClassifier
from sklearn.metrics import accuracy_score

model=OneVsRestClassifier(KNeighborsClassifier())

In [ ]:
model.fit(X_train, Y_train)
yPred=model.predict(X_test)
print(accuracy_score(Y_test, yPred))